In [43]:
import pandas as pd

df = pd.read_csv("관광데이터.csv", encoding="cp949")

In [44]:
processed_df = df.copy()

cols_to_process = ['food_percent', 'shopping_percent', 'accommodation_percent']
# for 문을 사용해 각 컬럼에 순차적으로 로직 적용
for col in cols_to_process:
    processed_df[col].fillna(0, inplace=True)
    
    print(f"\n'{col}' 컬럼 처리 완료. (대체값: {0})")

cols_to_compare = ['accommodation_percent', 'food_percent', 'shopping_percent']

max_col_names = processed_df[cols_to_compare].idxmax(axis=1)

value_map = {
    'accommodation_percent': 1,
    'food_percent': 2,
    'shopping_percent': 3
}

processed_df['main_expense'] = max_col_names.map(value_map)
#processed_df.drop(columns= cols_to_compare, inplace = True)

convert_dict_int = {
    'food': 'Int64',
    'landscape': 'Int64',
    'heritage': 'Int64',
    'main_expense': 'Int64',
    'language': 'Int64',
    'safety': 'Int64',
    'budget': 'Int64',
    'accommodation': 'Int64',
    'transport': 'Int64',
    'navigation': 'Int64',
}

processed_df = processed_df.astype(convert_dict_int)


convert_dict_wo_order = {
    'country': 'category',
    'main_expense': 'category',
    'planned_activity': 'category',
    'visit_local_indicator': 'category'
}

processed_df = processed_df.astype(convert_dict_wo_order)



'food_percent' 컬럼 처리 완료. (대체값: 0)

'shopping_percent' 컬럼 처리 완료. (대체값: 0)

'accommodation_percent' 컬럼 처리 완료. (대체값: 0)


C:\Users\SyngyeonTak\AppData\Local\Temp\ipykernel_73320\2381458765.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processed_df[col].fillna(0, inplace=True)


In [45]:
processed_df.drop(columns= cols_to_compare, inplace = True)


In [46]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32064 entries, 0 to 32063
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Unnamed: 0             32064 non-null  int64   
 1   ID                     32064 non-null  float64 
 2   year                   32064 non-null  int64   
 3   month                  32064 non-null  int64   
 4   quarter                32064 non-null  int64   
 5   country                32064 non-null  category
 6   gender                 32064 non-null  int64   
 7   age                    32064 non-null  int64   
 8   visit_purpose          32064 non-null  int64   
 9   num_visits             32064 non-null  int64   
 10  revisit_indicator      32064 non-null  int64   
 11  travel_type            32064 non-null  int64   
 12  satisfaction           32064 non-null  int64   
 13  revisit_intent         32064 non-null  int64   
 14  recommend_intent       32064 non-null 

In [47]:
nan_counts = processed_df.isnull().sum()

# NaN 개수가 0보다 큰 컬럼들만 필터링
columns_with_nan = nan_counts[nan_counts > 0]

print("--- NaN을 포함한 변수와 그 개수 ---")
print(columns_with_nan)
print(processed_df.columns)

--- NaN을 포함한 변수와 그 개수 ---
food             3503
landscape        6308
heritage         9821
language           37
safety             77
budget             84
accommodation      47
transport          42
navigation        779
dtype: int64
Index(['Unnamed: 0', 'ID', 'year', 'month', 'quarter', 'country', 'gender',
       'age', 'visit_purpose', 'num_visits', 'revisit_indicator',
       'travel_type', 'satisfaction', 'revisit_intent', 'recommend_intent',
       'visit_local_indicator', 'seoul', 'gyeonggi', 'incheon', 'gangwon',
       'chungcheong', 'gyeongsang', 'jeolla', 'jeju', 'planned_activity',
       'stay_duration', 'total_duration', 'total_expense',
       'accommodation_cost', 'food_cost', 'shopping_cost', 'food', 'landscape',
       'heritage', 'language', 'safety', 'budget', 'accommodation',
       'transport', 'navigation', 'main_expense'],
      dtype='object')


In [48]:
print(processed_df.columns)
columns_to_drop = ["Unnamed: 0", "ID", "year", "quarter", "visit_purpose"
    , "num_visits", "travel_type", "stay_duration"
    , "total_expense", "accommodation_cost", "food_cost", "shopping_cost"
    ]

processed_df.drop(columns= columns_to_drop, inplace = True)



Index(['Unnamed: 0', 'ID', 'year', 'month', 'quarter', 'country', 'gender',
       'age', 'visit_purpose', 'num_visits', 'revisit_indicator',
       'travel_type', 'satisfaction', 'revisit_intent', 'recommend_intent',
       'visit_local_indicator', 'seoul', 'gyeonggi', 'incheon', 'gangwon',
       'chungcheong', 'gyeongsang', 'jeolla', 'jeju', 'planned_activity',
       'stay_duration', 'total_duration', 'total_expense',
       'accommodation_cost', 'food_cost', 'shopping_cost', 'food', 'landscape',
       'heritage', 'language', 'safety', 'budget', 'accommodation',
       'transport', 'navigation', 'main_expense'],
      dtype='object')


In [49]:
processed_df.rename(columns={'planned_activity': 'cluster'}, inplace=True)

dist_columns = ['age', 'gender', 'revisit_indicator', 'visit_local_indicator', 'month', 'main_expense', 'country']
distributions = {}

for col in dist_columns:
    dist = (
        processed_df
        .groupby('cluster')[col]
        .value_counts(normalize=True)
        .mul(100)
        .round(2)
        .unstack(fill_value=0)
    )
    distributions[f'{col}_by_cluster'] = dist  # key 이름도 수정

region_columns = ['seoul', 'gyeonggi', 'incheon', 'gangwon',
                  'chungcheong', 'gyeongsang', 'jeolla', 'jeju']

region_kor_mapping = {
    'seoul': '서울',
    'gyeonggi': '경기',
    'incheon': '인천',
    'gangwon': '강원',
    'chungcheong': '충청',
    'gyeongsang': '경상',
    'jeolla': '전라',
    'jeju': '제주'
}

for col in region_columns:
    region_rate = (
        processed_df
        .groupby('cluster')[col]
        .mean()
        .mul(100)
        .round(2)
    )
    kor_col_name = region_kor_mapping[col] + '_방문률'
    distributions[kor_col_name + '_by_cluster'] = region_rate  # key 이름 명확히

mean_columns = ['satisfaction', 'total_duration']
means = {}

for col in mean_columns:
    stats = (
        processed_df
        .groupby('cluster')[col]
        .agg(['mean', 'std'])
        .round(2)
    )
    means[f'{col}_by_cluster'] = stats  # key 이름 수정

cluster_mapping = {'한류/공연': 1,'문화/역사/자연': 2,'음식/미식': 3,'쇼핑': 4,'스포츠/레저': 5,'뷰티/휴식/의료': 6,'유흥/오락': 7,'현대문화': 8,'기타': 99}
country_mapping = {'중국': 1, '일본': 2, '대만': 3, '미국': 4, '홍콩': 5, '태국': 6, '베트남': 7, '말레이시아': 8, '싱가포르': 9, '필리핀': 10, '중동': 11, '인도네시아': 12, '캐나다': 13, '호주': 14, '러시아': 15, '영국': 16, '독일': 17, '프랑스': 18, '몽골': 19, '인도': 20, '기타': 99}
visit_local_mapping = {'서울': 0, '서울+지방': 1, '지방': 2}
main_expense_mapping = {'숙박': 1, '음식': 2, '쇼핑': 3}
age_mapping = {'10대': 1, '20대': 2, '30대': 3, '40대': 4, '50대': 5, '60대이상':6}
gender_mapping = {'남성': 1, '여성': 2}

# --- 2. 조회용 역방향 딕셔너리 생성 ---
reverse_cluster_mapping = {v: k for k, v in cluster_mapping.items()}
reverse_country_mapping = {v: k for k, v in country_mapping.items()}
reverse_visit_local_mapping = {v: k for k, v in visit_local_mapping.items()}
reverse_main_expense_mapping = {v: k for k, v in main_expense_mapping.items()}
reverse_age_mapping = {v: k for k, v in age_mapping.items()}
reverse_gender_mapping = {v: k for k, v in gender_mapping.items()}

# cluster만 기준
all_keys = distributions['age_by_cluster'].index
cluster_ids = all_keys if isinstance(all_keys, pd.Index) else []

for cluster_id in cluster_ids:
    cluster_name = reverse_cluster_mapping.get(cluster_id, f"알 수 없는 군집({cluster_id})")

    print(f"--- 📊 프로필: 군집 [{cluster_name}] ---")

    print("\n[인구통계학적 특성]")
    try:
        age_series = distributions['age_by_cluster'].loc[cluster_id]
        age_series = age_series.sort_index()
        age_str = ", ".join([f"{reverse_age_mapping.get(age_id, age_id)}({pct:.2f}%)" for age_id, pct in age_series.items()])
        print(f"  - 연령대: {age_str}")

        gender_series = distributions['gender_by_cluster'].loc[cluster_id]
        gender_str = ", ".join([f"{reverse_gender_mapping.get(gender_id, gender_id)}({pct:.2f}%)" for gender_id, pct in gender_series.items()])
        print(f"  - 성별 분포: {gender_str}")

        country_series = distributions['country_by_cluster'].loc[cluster_id]
        country_str = ", ".join([
            f"{reverse_country_mapping.get(type_id, type_id)}({pct:.2f}%)"
            for type_id, pct in country_series.items()
        ])
        print(f"  - 국가 분포: {country_str}")
    except KeyError:
        print("  - 인구통계 정보 부족")

    print("\n[여행 행태]")
    try:
        revisit_series = distributions['revisit_indicator_by_cluster'].loc[cluster_id]
        revisit_pct = revisit_series.get(1, 0) if isinstance(revisit_series, pd.Series) else revisit_series
        print(f"  - 재방문 비율: {revisit_pct:.2f}%")

        local_visit_series = distributions['visit_local_indicator_by_cluster'].loc[cluster_id]
        local_visit_str = ", ".join([
            f"{reverse_visit_local_mapping.get(type_id, type_id)}({pct:.2f}%)"
            for type_id, pct in local_visit_series.items()
        ])
        print(f"  - 지방 방문 유형: {local_visit_str}")

        print("  - 주된 지출 항목 분포:")
        main_expense_series = distributions['main_expense_by_cluster'].loc[cluster_id]
        for expense_id, pct in main_expense_series.items():
            expense_str = reverse_main_expense_mapping.get(expense_id, f"유형{expense_id}")
            print(f"    - {expense_str}: {pct:.2f}%")

        print("\n[방문 월 분포]")
        month_series = distributions['month_by_cluster'].loc[cluster_id]
        month_series = month_series.sort_index()
        for month, pct in month_series.items():
            print(f"  - {month}월: {pct:.2f}%")
    except (KeyError, ValueError):
        print("  - 여행 행태 정보 부족")

    print("\n[지역 방문 비율]")
    for region in region_columns:
        key = f"{region_kor_mapping[region]}_방문률_by_cluster"
        try:
            pct = distributions[key].loc[cluster_id]
            print(f"  - {region_kor_mapping[region]} 방문률: {pct:.2f}%")
        except KeyError:
            continue

    print("\n[만족도 및 체류 기간]")
    try:
        satisfaction_stats = means['satisfaction_by_cluster'].loc[cluster_id]
        duration_stats = means['total_duration_by_cluster'].loc[cluster_id]
        print(f"  - 전체 만족도: 평균 {satisfaction_stats['mean']:.2f} (표준편차: {satisfaction_stats['std']:.2f})")
        print(f"  - 전체 체류 기간: 평균 {duration_stats['mean']:.2f}일 (표준편차: {duration_stats['std']:.2f})")
    except KeyError:
        print("  - 만족도/체류 기간 정보 부족")

    print("\n" + "="*50)

--- 📊 프로필: 군집 [한류/공연] ---

[인구통계학적 특성]
  - 연령대: 10대(6.73%), 20대(54.38%), 30대(19.23%), 40대(10.23%), 50대(7.56%), 60대이상(1.87%)
  - 성별 분포: 남성(16.52%), 여성(83.48%)
  - 국가 분포: 중국(18.79%), 일본(16.64%), 대만(9.36%), 미국(4.70%), 홍콩(8.84%), 태국(4.42%), 베트남(1.27%), 말레이시아(3.86%), 싱가포르(5.14%), 필리핀(7.01%), 중동(1.51%), 인도네시아(3.78%), 캐나다(2.35%), 호주(2.87%), 러시아(1.55%), 영국(1.91%), 독일(1.51%), 프랑스(1.04%), 몽골(0.52%), 인도(0.64%), 기타(2.31%)

[여행 행태]
  - 재방문 비율: 57.01%
  - 지방 방문 유형: 서울(69.31%), 서울+지방(26.51%), 지방(4.18%)
  - 주된 지출 항목 분포:
    - 숙박: 45.42%
    - 음식: 31.17%
    - 쇼핑: 23.41%

[방문 월 분포]
  - 1월: 8.24%
  - 2월: 5.33%
  - 3월: 7.09%
  - 4월: 7.09%
  - 5월: 9.32%
  - 6월: 9.32%
  - 7월: 10.67%
  - 8월: 10.67%
  - 9월: 6.41%
  - 10월: 11.03%
  - 11월: 7.64%
  - 12월: 7.21%

[지역 방문 비율]
  - 서울 방문률: 95.82%
  - 경기 방문률: 7.40%
  - 인천 방문률: 4.22%
  - 강원 방문률: 6.57%
  - 충청 방문률: 1.00%
  - 경상 방문률: 14.69%
  - 전라 방문률: 1.31%
  - 제주 방문률: 4.22%

[만족도 및 체류 기간]
  - 전체 만족도: 평균 4.59 (표준편차: 0.57)
  - 전체 체류 기간: 평균 6.87일 (표준편차: 5.47)

--- 📊 프로필: 

C:\Users\SyngyeonTak\AppData\Local\Temp\ipykernel_73320\4098954099.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('cluster')[col]
C:\Users\SyngyeonTak\AppData\Local\Temp\ipykernel_73320\4098954099.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('cluster')[col]
C:\Users\SyngyeonTak\AppData\Local\Temp\ipykernel_73320\4098954099.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('cluster

목표
1단계: 군집 프로파일링 (Cluster Profiling) - '고객 유형' 발견 
2단계: 페르소나 (Persona) - '전문가 캐릭터' 설정
3단계: LLM 파인튜닝 효과 (Effect) - '맞춤형 AI' 탄생

원하는 출력물
"해당 사용자는 A군집에 속한다. A군집은 주로 *어느나라가 많이 속하고 *체제기간은 얼마나 되고 ... *소비 패턴은 이러하다. 
이들은 지방을 여행할 때 주로 어느지역에 여행을 하며 이들이 지방에 감으로써 향상되는 만족도 항목은 X1, X2, X3 이며 저하되는 만족도 항목은 Y1, Y2, Y3 이다.
전체적인 만족도 향상에 영향을 준 항목 만족도는 X_alpha이고 만족도 하락에 영향을 준 항목 만족도는 Y_alpha 이다"





In [50]:
import pandas as pd
import xgboost as xgb
import shap
import matplotlib.pyplot as plt

# ------------------- 🔧 1. 기본 설정 및 데이터 준비 🔧 -------------------
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# (processed_df, reverse_cluster_mapping 등은 이미 있다고 가정)
# processed_df = pd.read_csv(...)
# reverse_cluster_mapping = {0: '한류/공연', 1: '문화/역사/자연', ...}

seoul_comparison_df = processed_df.copy()

satisfaction_details = ['food', 'landscape', 'heritage', 'language', 'safety',
                        'budget', 'accommodation', 'transport', 'navigation']
region_columns = ['seoul', 'gyeonggi', 'incheon', 'gangwon',
                  'chungcheong', 'gyeongsang', 'jeolla', 'jeju']
categorical_columns = ['cluster', 'main_expense', 'country']

region_kor_mapping = {
    'seoul': '서울', 'gyeonggi': '경기', 'incheon': '인천',
    'gangwon': '강원', 'chungcheong': '충청', 'gyeongsang': '경상',
    'jeolla': '전라', 'jeju': '제주'
}
# ------------------- ⚙️ 2. 모델 학습 및 SHAP 값 계산 ⚙️ -------------------
X = seoul_comparison_df.drop(columns='satisfaction')
X = pd.get_dummies(X, columns=categorical_columns, dummy_na=False)
y = seoul_comparison_df['satisfaction']

valid_index = y.dropna().index
X = X.loc[valid_index]
y = y.loc[valid_index]

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42, enable_categorical = True)
model.fit(X, y)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# ✅ SHAP 결과를 모든 feature에 대해 만듭니다. (만족도 항목 + 지역 항목 모두 포함)
shap_df = pd.DataFrame(shap_values, columns=X.columns, index=X.index)

# ------------------- 🗺️ 3. 항목별 최적 지역 분석 및 리포트 🗺️ -------------------
# 분석에 필요한 원본 데이터 컬럼과 SHAP 결과를 모두 합칩니다.
analysis_df = seoul_comparison_df.loc[X.index, ['cluster'] + satisfaction_details + region_columns].join(shap_df, rsuffix='_shap')

# 군집별 리포트 생성
for cluster_id, cluster_df in analysis_df.groupby('cluster'):
    cluster_name = reverse_cluster_mapping.get(cluster_id, f"군집 {cluster_id}")
    print(f"\n\n==============================================================")
    print(f"--- 📊 리포트: 군집 [{cluster_name}] ---")
    print(f"==============================================================")

    other_regions = [col for col in region_columns if col != 'seoul']
    
    # '서울 Only' 그룹 정의
    seoul_only_mask = (cluster_df['seoul'] == 1) & (cluster_df[other_regions].sum(axis=1) == 0)
    seoul_only_df = cluster_df[seoul_only_mask]

    if len(seoul_only_df) < 10:
        print("\n[정보] '서울 Only' 방문 그룹이 작아 분석을 건너뜁니다.")
        continue

    # ✅ [핵심 로직] 각 만족도 항목별로, SHAP 값을 가장 크게 높이는 추가 지역을 찾습니다.
    item_to_best_region = {}
    
    # 1. 각 만족도 항목(food, landscape...)에 대해 반복
    for item in satisfaction_details:
        item_shap_col = item + '_shap' # 해당 항목의 SHAP 컬럼명
        
        # '서울 Only' 그룹의 해당 항목 평균 SHAP 값을 기준점으로 설정
        base_item_shap = seoul_only_df[item_shap_col].mean()
        
        best_region_info = {'name': None, 'shap_lift': -99}

        # 2. 추가할 수 있는 다른 지역(강원, 제주...)에 대해 반복
        for region in other_regions:
            # '서울+지역' 그룹 정의
            seoul_plus_region_mask = (cluster_df['seoul'] == 1) & (cluster_df[region] == 1)
            seoul_plus_region_df = cluster_df[seoul_plus_region_mask]

            if len(seoul_plus_region_df) < 10: continue

            # '서울+지역' 그룹의 해당 항목 평균 SHAP 값
            compare_item_shap = seoul_plus_region_df[item_shap_col].mean()
            
            # 기준점 대비 SHAP 값 상승분(lift) 계산
            shap_lift = compare_item_shap - base_item_shap
            
            # 3. 이 상승분이 이전에 찾은 것보다 크면, '최적의 추가 지역'으로 기록
            if shap_lift > best_region_info['shap_lift']:
                best_region_info['name'] = region
                best_region_info['shap_lift'] = shap_lift
        
        # 4. 모든 지역을 비교한 후, 최종적으로 가장 좋았던 지역을 저장
        if best_region_info['name'] and best_region_info['shap_lift'] > 0:
            item_to_best_region[item] = best_region_info
            
    # 5. 최종 결과 리포트 출력
    if not item_to_best_region:
        print("\n[정보] '서울 Only' 대비 의미 있는 기여도 상승을 보이는 지방 조합을 찾지 못했습니다.")
    else:
        print("\n🏆 **'서울 Only' 대비, 항목별 만족도 기여도를 가장 크게 높이는 추가 지역:**")
        for item, info in item_to_best_region.items():
            region_kor = region_kor_mapping.get(info['name'], info['name'])
            print(f"  - **{item.capitalize()}** 기여도 최적화 지역: **{region_kor}** (기여도 +{info['shap_lift']:.3f} 상승)")

C:\Users\SyngyeonTak\AppData\Local\Temp\ipykernel_73320\3435677677.py:50: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for cluster_id, cluster_df in analysis_df.groupby('cluster'):




--- 📊 리포트: 군집 [한류/공연] ---

🏆 **'서울 Only' 대비, 항목별 만족도 기여도를 가장 크게 높이는 추가 지역:**
  - **Food** 기여도 최적화 지역: **제주** (기여도 +0.009 상승)
  - **Landscape** 기여도 최적화 지역: **인천** (기여도 +0.016 상승)
  - **Heritage** 기여도 최적화 지역: **제주** (기여도 +0.003 상승)
  - **Language** 기여도 최적화 지역: **전라** (기여도 +0.009 상승)
  - **Safety** 기여도 최적화 지역: **전라** (기여도 +0.014 상승)
  - **Budget** 기여도 최적화 지역: **제주** (기여도 +0.006 상승)
  - **Accommodation** 기여도 최적화 지역: **전라** (기여도 +0.012 상승)
  - **Transport** 기여도 최적화 지역: **전라** (기여도 +0.022 상승)
  - **Navigation** 기여도 최적화 지역: **전라** (기여도 +0.002 상승)


--- 📊 리포트: 군집 [문화/역사/자연] ---

🏆 **'서울 Only' 대비, 항목별 만족도 기여도를 가장 크게 높이는 추가 지역:**
  - **Food** 기여도 최적화 지역: **전라** (기여도 +0.001 상승)
  - **Landscape** 기여도 최적화 지역: **전라** (기여도 +0.011 상승)
  - **Heritage** 기여도 최적화 지역: **경기** (기여도 +0.004 상승)
  - **Language** 기여도 최적화 지역: **충청** (기여도 +0.004 상승)
  - **Safety** 기여도 최적화 지역: **전라** (기여도 +0.004 상승)
  - **Accommodation** 기여도 최적화 지역: **경상** (기여도 +0.001 상승)
  - **Transport** 기여도 최적화 지역: **전라** (기여도 +0.006 상승)
  - *

250717 정리

1. 군집 by 희준
군집 프로파일링

2. 군집별 항목 만족도 별 지역 방문 추천
항목 만족도 별 - (추천 지역, 상승 기여도)

3. 데이터 for 파인튜닝
- 군집 기술 (from 프로파일링)
- 입력 데이터
* 군집 평균에 비해 높은 항목 만족도 -> 실제 방문 지역 추천
* 군집 평균에 비해 낮은 항목 만족도 -> 군집 항목별 지역 추천 (기여도 순) 

여행자 id: 30
{
    군집: A
    군집 기술: 소속 군집은 A로 주로 쇼핑이 목적입니다. 여행자 id-30 의 국적/문화권에 소속 비중은 30%입니다.
    ... 

    id-30 여행객의 구체적인 정보
    {
        국적: x
        연령대: x
        방문 행태: {...}
        고려 사항: {...}
        ...
    }

    실제 방문: ['서울-명동']
    군집 평균에 비해 높은 항목 만족도: ['음식', '교통', '숙박']
    군집 평균에 비해 낮은 항목 만족도: ['자연경관', '역사/고궁']
        ['자연경관', '역사/고궁'] 만족도를 높일 수 있는 지역 
            = A 군집 자연경관 기여 지역 ['전라']
            = A 군집 역사/고궁 기여 지역 ['경상']

    최종 추천 지역 
    * 실제 방문 지역 ['서울-명동'] - 연관 항목 만족도 ['음식', '교통', '숙박']
    * 군집 추천 지역 ['전라', '경상'] - 연관 항목 만족도 ['자연경관', '역사/고궁']
}

추가 고려 사항
1. 지방만 여행한 사람 -> 지방 + 서울 (서울 여행을 하였을 때 어느 항목에서 만족도가 높았는 지)
    단점: 대신 서울 방문 뿐만 아니라 다른 지역 방문을 유치하는 취지와는 맞지 않을 수 있음

본 프로젝트의 장점
1. 다년간 이종 데이터의 통합 및 분석 기반 구축
    (데이터 정제 및 통합) 각기 다른 변수 구조를 가진 2018, 2019, 2023, 2024년의 데이터를 공통 변수 기준으로 정제하고 통합하여, 연도별 데이터의 차이를 극복한 대규모 분석 데이터셋을 성공적으로 구축했습니다. 이는 데이터 핸들링의 기술적 어려움을 해결했다는 점에서 큰 차별성을 가집니다.

2. 데이터 기반의 객관적 방한객 군집 도출
    (설득력 있는 군집화) 통합된 대규모 데이터를 기반으로 PCA와 클러스터링을 적용하여, 직관이나 가정이 아닌 데이터에 근거한 객관적이고 설득력 있는 핵심 방한객 그룹을 정의했습니다. 

3. SHAP(XAI) 기반의 해석 가능한 추천 모델 구현
    (설명력 있는 추천) 단순히 "A를 추천합니다"라는 결과를 제시하는 '블랙박스' 모델에서 나아가, SHAP 분석을 통해 각 군집의 특성과 추천의 근거를 설명할 수 있도록 모델을 구축했습니다. 이는 "왜 이 지역을 추천하는지"에 대한 명확한 이유를 제공하여, 최종 추천 결과의 신뢰도와 설득력을 극대화하는 핵심적인 장점입니다.

4. 최신 데이터를 활용한 추천 알고리즘의 고도화
    (심층적 행태 분석) 안정적인 군집화는 통합 데이터를 활용했지만, LLM 파인튜닝을 위한 최종 추천 알고리즘에는 최신 2024년 데이터에만 존재하는 풍부하고 세분화된 변수까지 적극 활용했습니다. 이를 통해 최신 트렌드와 방문객의 구체적인 행태를 반영한, 한 차원 높은 수준의 개인화 추천을 구현했습니다.

